# Install and Import Modules

In [2]:
!pip install easyocr

  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached python_bidi-0.6.3-cp310-none-win_amd64.whl.metadata (5.0 kB)
  Using cached shapely-2.0.6-cp310-cp310-win_amd64.whl.metadata (7.2 kB)
  Using cached pyclipper-1.3.0.post6-cp310-cp310-win_amd64.whl.metadata (9.2 kB)
Using cached easyocr-1.7.2-py3-none-any.whl (2.9 MB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 16.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   -- ------------------------------------- 12.8/203.1 MB 62.0 MB/s eta 0:00:04
   --- ------------------------------------ 17.6/203.1 MB 42.6 MB/s eta 0:00:05
   --- ------------------------------------ 18.4/203.1 MB 30.5 MB/s eta 0:00:07
   --- ------------------------------------ 19.1/203.1 MB 23.7 MB/s eta 0:00:08
   --- ---------------------

In [14]:
!pip install pandas
!pip install datasets
!pip install -U scikit-learn scipy matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------------------ --------------------- 5.2/11.1 MB 29.1 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 34.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 8.0/8.0 MB 45.4 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 41.6 MB/s eta 0:00:00


In [56]:
!pip install kagglehub

In [81]:
import easyocr
import time
import pandas as pd
import kagglehub
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from typing import List
import string

reader = easyocr.Reader(["en"])
path = kagglehub.dataset_download("trainingdatapro/ocr-receipts-text-detection")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


# Define necessary functions

In [101]:
def normalize_text(text: str) -> str:
    """
    Normalize text by converting to lowercase and stripping punctuation.
    """
    return text.lower().translate(str.maketrans('', '', string.punctuation))

In [102]:
def calculate_accuracy(recognized: List[str], valid: List[str]) -> dict:
    recognized_set = {normalize_text(word) for word in recognized}
    valid_set = {normalize_text(word) for word in valid}
    
    true_positives = len(recognized_set & valid_set)
    false_positives = len(recognized_set - valid_set)
    false_negatives = len(valid_set - recognized_set)
    
    precision = true_positives / (true_positives + false_positives) if recognized_set else 0
    recall = true_positives / (true_positives + false_negatives) if valid_set else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0
    
    return {
        "precision": precision * 100,
        "recall": recall * 100,
        "f1_score": f1_score * 100,
        "accuracy": (true_positives / len(valid)) * 100 if valid else 0
    }

# Define valid data array

In [103]:
valid_data = [
    ['BANANAS', 'FRAP'],
    ['R-CARROTS SHREDDED 10 OZ', 'R-CUCUMBERS PERSIAN 1 LB', 'TOMATOES CRUSHED NO SALT', 'TOMATOES WHOLE NO SALT W/BASIL', 
     'ORGANIC OLD FASHIONED OATMEAL', 'MINI-PEARL TOMATOES..', 'PKG SHREDDED MOZZARELLA LITE T', 'EGGS 1 DOZ ORGANIC BROWN.', 
     'BEANS GARBANZO', 'SPROUTED CA STYLE', 'A-AVOCADOS HASS BAG 4CT', 'A-APPLE BAG JAZZ 2 LB', 'A-PEPPER BELL EACH XL RED', 
     'GROCERY NON TAXABLE', 'BANANAS ORGANIC', 'CREAMY SALTED PEANUT BUTTER', 'WHL WHT PITA BREAD', 'GROCERY NON TAXABLE'],
    ['GV OATMEAL', 'OT 200Z TUM', 'M ATHLETICS', 'DEXAS 15X20'],
    ['TATER TOTS', 'HARD/PROV/DC', 'SNACK BARS', 'HRI CL CHS', 'HRI CL CHS', 'HRI CL CHS', 'HRI 12 U SG', 'HRI CL PEP', 'EARBUDS',
      'SC BCN CHDDR', 'ABF THINBRST', 'ABF THINBRST', 'HARD/PROV/DC', 'DV RSE OIL M', 'APPLE 3 BAG', 'STOK LT SWT', 'PEANUT BUTTER',
      'AVO VERDE', 'ROLLS', 'BTS DRY BLON', 'GALE', 'TR HS FRM 4', 'BAGELS', 'GV SLIDERS', 'ACCESSORY', 'CHEEZE IT', 'RITZ', 'RUFFLES', 'GV HNY GRMS'],
    ['SW HRO FGHTR'],
    ['KTTL SEA SALT POT CHP', 'BRAIDED BRIOCHE', 'CHEF PLATE MEAL'],
    ['Woman', 'Ham Cheese', 'Ice Java Tea', 'Mineral Water', 'Black & White'],
    ['6 WING PLATE', 'ASST 27', 'CUTIE CAR'],
    ['FF BS BREAST', 'KS DICED TOM', 'JACKORGSALSE', '18CT EGGS', 'GRAPE TOMATO', 'ECO HALF PAN', 'GRND TURKEY', 'CHPD ONION', 'MONT JACK 2#'],
    ['N/YORK TX GRLC', 'CHICKEN WINGS', 'PICSWEET BLEND', 'PICSWEET GREENS', 'CHICKEN WINGS', 'DIET LN GRND B', 'PORK LOIN CHOP', 'DRUMSTICKS',
     'EYE RND STK FP', 'EYE RND STK FP', 'BEEF FOR STEW', 'PORK 1/2 LOIN', 'B/BEST S STEAK', 'BAR S MEAT FRNK', '0/M HAM', 'WINCO NOODLES', 'TNDRBRD CHIX BR'
     'KRFT DELUXE MAC', 'EGGO WAFFLE', 'L/D NUTTY BARS', 'L\D OATMEAL CRM', 'LINKS MILD', 'KEEB TOWNHOUSE', 'TURKEY FRANK', 'RESERS POT SLD', 'DM SPAG SAUCE',
     'HUNTS MANWICH', 'DM PNAPL CHNKY', 'SUNBEAN BUNS', 'WINCO SNDWCH WT', 'WINCO HOMO MILK', 'EGGS'],
    ['LAZENBY WORCESTER SAUCE', 'MILKY BAR CHOC', 'SMOKED VIENNAS', 'PEALED PEACHES', 'MEDITERRANEAN MIX', 'SPAR COOKING OIL',
     'F/L ENGLISH CUCUMB', 'NESTLE AERO', 'CADBURY DAIRY MI', 'GRAPES MIXED TUB', 'TASTIC RICE', 'BLACK CAT SMOOTH', 'CARRIER BAG 24L', 'BANANAS LOOSE'],
    ["PL TORTILLA'S", 'CAGE FREE ALL WHIT', 'BLACK BEANS', 'Frozen Mangoes 16o', 'Whole Strawberries', 'OG LF COTTAGE CHEE', 'MAHI MAHI FILLETS', '$2 off (1) WC Fill',
     'California Harvest', 'PLUMS BLACK CV'],
    ['SW FIGURES', 'TOOTHBRUSH', 'WOMEN SLIPPE'],
    ['GIFT CARD'],
    ['EQUATE LINER', 'VFUS ENG POM', 'DAWN ORIG', '50YD PKGTAPE', '50YD PKGTAPE', 'LINT POLLER2'],
    ['DIABETES'],
    ['6 WING PLATE', 'ASST 27', 'CUTIE CAR'],
    ['BANANAS', 'BEVERAGE', 'OS CRAN POM', 'STRWBRY CC', 'CAMPARI TOM', 'KFT SINGLES', 'HARD SALAMI', 'AVOCADO', 'PILLS WHITE', 'SH NYLON COL', 'HAND CLEANER', 'INJECTR CLNR'],
    ['GV WATER', 'GV WATER', 'GREAT VALUE', 'GREAT VALUE', 'GAIN LFE', 'GAIN', 'GV VEG OIL', 'SNICKER FS', 'GDBR FS 6PK', 'MIXED CHOC', 'CHERRIES', 'GAIN', 'HY PSTLEG PM',
     'BATTERIES', 'GV 20Z MINI', 'CRNBRY SAUCE', 'SPREADS', 'EGGS 6CT'],
    ['GRILL COVER', 'FIBER CHOICE', 'CELERY HEART', 'RED GRAPE'],
]

# Recognize dataset images. Output metricies

In [ ]:
for i in range(len(valid_data)):
    start_time = time.perf_counter()
    
    result = reader.readtext(f'{path}/images/{i}.jpg', detail=0)
    metrics = calculate_accuracy(result, valid_data[i])
    
    end_time = time.perf_counter()
    print(f'Image {i}:')
    print(f"  Precision: {metrics['precision']:.2f}%")
    print(f"  Recall: {metrics['recall']:.2f}%")
    print(f"  F1-Score: {metrics['f1_score']:.2f}%")
    print(f"  Accuracy: {metrics['accuracy']:.2f}%")
    print(f"  Time: {end_time - start_time}")
    print('-' * 30)

Image 0:
  Precision: 2.08%
  Recall: 50.00%
  F1-Score: 4.00%
  Accuracy: 50.00%
  Time: 4.3314725999953225
------------------------------
Image 1:
  Precision: 5.45%
  Recall: 17.65%
  F1-Score: 8.33%
  Accuracy: 16.67%
  Time: 4.617287999950349
------------------------------
Image 2:
  Precision: 4.92%
  Recall: 75.00%
  F1-Score: 9.23%
  Accuracy: 75.00%
  Time: 3.7626693999627605
------------------------------
Image 3:
  Precision: 4.55%
  Recall: 16.00%
  F1-Score: 7.08%
  Accuracy: 13.79%
  Time: 5.442208300111815
------------------------------
Image 4:
  Precision: 0.00%
  Recall: 0.00%
  F1-Score: 0.00%
  Accuracy: 0.00%
  Time: 2.2457077000290155
------------------------------
Image 5:
  Precision: 1.16%
  Recall: 33.33%
  F1-Score: 2.25%
  Accuracy: 33.33%
  Time: 19.07333819998894
------------------------------
Image 6:
  Precision: 6.38%
  Recall: 60.00%
  F1-Score: 11.54%
  Accuracy: 60.00%
  Time: 9.155606399988756
------------------------------
Image 7:
  Precision: 1.5